In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../src/utils")
from utils import SimpleGraphVoltDatasetLoader_Lazy,read_and_prepare_data,read_raw_network_data,preprocess_nodes,fill_data_simple_homogeneous,preprocess
import networkx as nx
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import random
from matplotlib.colors import LinearSegmentedColormap
from plotly.subplots import make_subplots
import colorsys

In [ ]:
trafo_id = "T1330"
num_timesteps_in = 12
num_timesteps_out = 4

## Testing

In [ ]:
raw_data,_ = read_raw_network_data(trafo_id)

In [ ]:
raw_data = fill_data_simple_homogeneous(raw_data)

In [ ]:
data = preprocess(raw_data)

In [ ]:
#transform raw_data["nodes_static_data"] to a list ordered by node_id and put x_y coordinates to the list
nodes_static_data = data["nodes_static_data"]
nodes_static_data = nodes_static_data.sort_values(by=['node_id'])
nodes_static_data = nodes_static_data.reset_index(drop=True)
nodes_coords = nodes_static_data[["x_y"]]
nodes_coords = nodes_coords.values.tolist()

#this is it now ['(530670.5350000001, 153984.66459999979)'] now we need to split the string and convert to float

nodes_coords = [x[0].split(",") for x in nodes_coords]
nodes_coords = [[float(x[0][1:]),float(x[1][:-1])] for x in nodes_coords]

In [ ]:
df_unique_nodes = nodes_static_data.drop_duplicates(subset='node_id')
result_df = df_unique_nodes[df_unique_nodes['aclass_id'] != 'PMO']
not_pmo_nodes = result_df['node_id'].tolist()

#i want to add some hardcoded nodes to the list
not_pmo_nodes.append(47)

In [ ]:
G = nx.from_pandas_edgelist(data['edges_static_data'],source='from_node_id',target='to_node_id')

In [ ]:
node_colors = ["grey" if node in not_pmo_nodes else "skyblue" for node in G.nodes]

# Draw the graph with specified node colors
plt.figure(figsize=(10, 10))
nx.draw(G, with_labels=True, node_size=500, node_color=node_colors, edge_color="grey")
plt.show()

In [ ]:
weights_2d = [np.abs([round(random.random(), 2) for _ in range(len(G.nodes()))]) for _ in range(100)]

In [ ]:
#for each first dim change the second dim on index from not_pmo_nodes to nan
for i in range(len(weights_2d)):
    for j in range(len(weights_2d[i])):
        if j in not_pmo_nodes:
            weights_2d[i][j] = np.nan

## new

In [ ]:
from utils import SimpleGraphVoltDatasetLoader_Lazy, read_and_prepare_data
from torch_geometric_temporal.signal import temporal_signal_split
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN, GConvLSTM
from tqdm import tqdm

import sys
sys.path.append("../src/utils")
from models import avgModel, RNN_LSTM, GNN_A3TGCN,GNN_GCNLSTM
sys.path.append("../src/visualization")
from visualize import plot_network_errors

In [ ]:
model_name = 'GNN_GCNLSTM_20231217035852_T1330_epochs-50_in-12_out-4_train-ratio-1month_lr-0.01_hidden-64.pt'
trafo_id = "T1330"
num_timesteps_in = 12
num_timesteps_out = 4
train_ratio = 0.7
test_ratio_vs_eval_ratio = 0.5
device = torch.device('cpu')

In [ ]:
print("Loading data...")
loader = SimpleGraphVoltDatasetLoader_Lazy(trafo_id, num_timesteps_in, num_timesteps_out)
loader_data_index = loader.snapshot_index


In [ ]:
train_dataset, test_eval_dataset = loader.temporal_signal_split_lazy_cut(loader_data_index)
test_dataset, eval_dataset = loader.temporal_signal_split_lazy(test_eval_dataset, train_ratio=test_ratio_vs_eval_ratio)

In [ ]:
#read saved model
model = GNN_GCNLSTM(node_features=loader.num_features, periods=num_timesteps_out,hidden=64).to(device)
model.load_state_dict(torch.load(f"../models/final/{model_name}", map_location=torch.device('cpu')))

In [ ]:
is_pmo = np.ones((113,4))

#if the node is not pmo then set the value to 0
for i in range(len(is_pmo)):
    if i in not_pmo_nodes:
        is_pmo[i] = 0
    

In [ ]:
std = loader.mean_and_std["measurements"][1]["voltage"]
mean = loader.mean_and_std["measurements"][0]["voltage"]
num_of_nodes = loader.get_snapshot(0).x.shape[0]

In [ ]:
#number of zeros in is_pmo
num_pmo = np.count_nonzero(is_pmo) / 4

In [ ]:
#create a np array of zeros for each node
predsLSTM = np.zeros((num_of_nodes, len(eval_dataset)))
truthLSTM = np.zeros((num_of_nodes, len(eval_dataset)))

loss_elementwiseLSTM = np.zeros((0, num_timesteps_out))
loss_fn = torch.nn.L1Loss

model.eval()
with torch.no_grad():
    index = 0
    for snapshot_j in tqdm(eval_dataset, desc="Evaluating model"):
        
        snapshot = loader.get_snapshot(snapshot_j)
        snapshot = snapshot.to(device)
        pred = model(snapshot.x, snapshot.edge_index)
        pred = pred.detach().numpy()
        y = snapshot.y.detach().numpy()
        truthLSTM[:, index] = y[:, 0]*std+mean
        predsLSTM[:, index] = pred[:, 0]*std+mean
        
        pred = torch.from_numpy(pred * is_pmo)
        y = torch.from_numpy(y * is_pmo)

        loss = loss_fn(reduction="none")(pred, y).cpu().numpy()

        mask = np.ones(loss.shape[0], dtype=bool)
        mask[not_pmo_nodes] = False
        result_array = loss[mask]

        #print(result_array.shape)

        for i in range(result_array.shape[0]):
            #print(i)
            for j in range(result_array.shape[1]):
                if result_array[i][j]*std > 50:
                    print(i)

        loss_elementwiseLSTM = np.concatenate((loss_elementwiseLSTM, result_array), axis=0)
                    
        index += 1

    loss_elementwiseLSTM *= std


In [ ]:
#read saved model
model_name2 = 'GNN_A3TGCN_20231217041452_T1330_epochs-200_in-12_out-4_train-ratio-1month_lr-0.01_hidden-64.pt'

model2 = GNN_A3TGCN(node_features=loader.num_features, periods=num_timesteps_out,hidden=64).to(device)
model2.load_state_dict(torch.load(f"../models/final/{model_name2}", map_location=torch.device('cpu')))

In [ ]:
#create a np array of zeros for each node
predsA3 = np.zeros((num_of_nodes, len(eval_dataset)))
truthA3 = np.zeros((num_of_nodes, len(eval_dataset)))

loss_elementwiseA3 = np.zeros((0, num_timesteps_out))
loss_fn = torch.nn.L1Loss

model2.eval()
with torch.no_grad():
    index = 0
    for snapshot_j in tqdm(eval_dataset, desc="Evaluating model"):
        
        snapshot = loader.get_snapshot(snapshot_j)
        snapshot = snapshot.to(device)
        pred = model2(snapshot.x, snapshot.edge_index)
        pred = pred.detach().numpy()
        y = snapshot.y.detach().numpy()
        truthA3[:, index] = y[:, 0]*std+mean
        predsA3[:, index] = pred[:, 0]*std+mean

        pred = torch.from_numpy(pred * is_pmo)
        y = torch.from_numpy(y * is_pmo)

        loss = loss_fn(reduction="none")(pred, y).cpu().numpy()

        mask = np.ones(loss.shape[0], dtype=bool)
        mask[not_pmo_nodes] = False
        result_array = loss[mask]

        #print(result_array.shape)

        for i in range(result_array.shape[0]):
            #print(i)
            for j in range(result_array.shape[1]):
                if result_array[i][j]*std > 50:
                    print(i)
                    
        loss_elementwiseA3 = np.concatenate((loss_elementwiseA3, result_array), axis=0)
                    
        index += 1

    loss_elementwiseA3 *= std


In [ ]:
loss_elementwiseLSTM

In [ ]:
loss_elementwiseLSTM.shape

In [ ]:
loss_elementwiseA3

In [ ]:
loss_elementwiseA3.shape

In [ ]:
import plotly.graph_objects as go

# Reshape the arrays to create separate boxes for each timestamp
data_array1_reshaped = loss_elementwiseLSTM.T
data_array2_reshaped = loss_elementwiseA3.T

# Create Box plots using Plotly
fig = go.Figure()

def darken_color(rgb, factor):
    h, l, s = colorsys.rgb_to_hls(rgb[0] / 255, rgb[1] / 255, rgb[2] / 255)
    l *= factor
    r, g, b = colorsys.hls_to_rgb(h, l, s)
    return int(r * 255), int(g * 255), int(b * 255)


# Add Box plot for the first array
for i in range(loss_elementwiseLSTM.shape[1]):

    color1 = color = darken_color((0, 0, 255), 1 - i * 0.2)
    color2 = color = darken_color((255, 165, 0), 1 - i * 0.2)
    
    fig.add_trace(
        go.Box( y=data_array1_reshaped[i], 
                name=f'LSTM - Timestamp {i+1}', 
                marker_color=f'rgb{color1}'))
    fig.add_trace(
        go.Box( y=data_array2_reshaped[i], 
                name=f'A3GCN - Timestamp {i+1}',
                marker_color=f'rgb{color2}'))


# Update layout
fig.update_layout(title='Box Plot for Two Models',
                  xaxis=dict(title='Timestamp'),
                  yaxis=dict(title='Error'))

# Show the plot
fig.show()


In [ ]:
#calculate the mean of the loss for the two models
mean_loss1 = np.mean(loss_elementwiseLSTM,axis=0)
mean_loss2 = np.mean(loss_elementwiseA3,axis=0)

print(f"Mean loss LSTM: {mean_loss1}")
print(f"Mean loss A3TGCN: {mean_loss2}")

print(f"Mean loss LSTM: {np.mean(mean_loss1)}")
print(f"Mean loss A3TGCN: {np.mean(mean_loss2)}")